# Digital services 


For many decades researchers and industry tried to build a secure and private digital services. For example, a protocl that allow to interact with a bank in anonymous way. Such solution makes sense when:
- Individuals rely on the regulatory enviroment 
- When there is a threat when illegal action is observed 

How can you build such service over the internet? What if your service is in another country? In a mission-critical/Byzantine enviorment? 

## Decentralized enviorments 

The key challenge is to co-ordinate number of individuals even in the absence of a strong regulatory enviorment. 
Beacuse we cannot rely on any particular individual that makes such system highly robust and accessable to anyone having a connection.  

However building such services over decentrlized evnoirments introudces some number of challenges that are not present in central services:

1. All participants are usually not fully connected. All communication is through a "communication graph". How to build a robust communication graph? 
2. Processes are allowed to be offline. Opt-out and rejoin later to the system 
3. There are financial incentives to behave in many non-trivial ways. *Example*


To work in such envoirment one must first assume the maximum number of malicious participants in the network. 
$f$


More challenges: 
- You cannot rely on certain peer being always honest - i.e. executing the same code. 
- Achieving privacy is harder


## Bitcoin model 

Achieves a distributed consensus among individuals despite challenges. 
The consensus is split in rounds. At each round peers collect transactions and append a list of *non-confirmed* transactions to the list of all transactions. Such list of transaction is usually called *block*. 

Each transaction goes through a validation to get into the *block*. For example, for a payment system, the validation protocol ensures that balance of the payer is not negative. 

*Ledger* - state at time $t$. To calculate state peer goes through all ledgers up to the current time and applies transactions. 


**Example**
Alice at round i wants to pay to Bob. Alice signs a message $(pay, Alice, Bob, 1, id)_{Bob}$
Alice gossips to the neighbours the transaction. So that this transaction is spread through the network. 
At the end to the round i a random leader is elected - say Charlie.

If Charlie is honest, he will include Alice's transaction to the newest block and links the block to the latest known block, thus forming a chain and a new ledger $L_i = B_i -> B_{i-1} -> ... -> L_0$  

That indicates that a random peer in the network has received and verified the transaction. Is that enough? 


Security of the Bitcoin comes from the following: 
 - Charlie cant make Bob or Alice pay, because each transaction requires a signature to get accepted. 
 - In worst case Charlie can ignore the transaction. If we assume that $f < N/2$, Alice will progress at least in half of the cases. 
 
**Assumptions**:
- Cryptography assumption 
- Unbiased random leader selection that is fair 
  


## Payment  systems

The problem with a currency in real life is that they are unforgable and difficult to replicate. Usually it is a physical object acting as a marker with some value. The value has only nominal meaning and the value is creating dure to belief that this currency can be spend later. In digital and decentralized space creating such marker is an hard problem, mainly due to the "copying" problem of the marker. One can reuse the same token with multiple peer knowing that these peers are not communicating with each other.

We can formalize payment system with a "Marker problem".

1. Participants can hold marker
2. When marker is transfered the holder loses it.
3. Receiver knows the Sender when the marker is received

Security guarantees: 
1. At most one process should be marked
2. If honest peer received it from another honest peer - the event happend
3. If peer received a chain ending with P - it is now marked

Complexities: 
1. Message Complexity per Round
2. Signature complexity per Round
3. Round complexity - number of steps taken



## Probablistic 

Probablistic payment network 
Rely on intersection and birthday paradox. 
Choose the peers that will prevent $P_1$ from double spending. 

Without using it build a determenisitc network - when the process itself already saw that it was spended. 


## Determensitic payment networks 

Connect in a cycle 
Require multiple messages per round. 

## Third pary trust 

Using intermediaries to build trust chanelles for inderection, but having following guarantees: 
1. The cheating is detectable, and there is a guaranteed bounded loss on it
2. Grim Trigger once cheating is detected, optionally a reputaiton loss. 
3. There are many intermediares in the market. 


#### Conserns 

- Discrete payments with nominal values vs Numerical values in the payment 
- Both sender and receiver should be honest 
- Unauthenticated communication in the payment systems
- Fault tolerant model, trust reobustness 


#### Secure Byzantine Broadcast

1. Designate $3f+1$ broadcast processes 
2. Broadcast the message to the procecesses 
3. The consensus processes send to $P_n$ marker 
4. If $P_n$ received at least $2f+1$ it is now marked. 





### Clean-slate design for the Decentralized Ledger 

1. Define participants and particiapnts roles 
2. 



- Events and validity of the events:

Present all transactions as events - split it with $k$ parties.



 * Event: Transaction signed by the peer A 
 * Event: Transaction signed by the peer B 


- 
 
**Overlay network** 

The key abstraction is the overlay network. 
They can be build or they can emerge themself.

Function is build to communicate over the overlay network;

Gossip protocols over the network:
 - Dissemination protocol, computation, overlay construction 
 
 - Application for distributed mining: 
  * EM algorithm, clustering, collobarative filtering 
  
Simple locally and emergent and powerful globally 

Epidemic Database Updates:
 - All databases must be the same in the face of updates, removals and additions 
 - Updates are local and are relicated among other nodes
 
Gossip based replication: 
- Spreading of the information 
- Error correction - anti-entropy gossip 

Anti-Entropy - guarntees perfect dissemination 

Rumor Mongering - Susceptible Infective Removed 

Push model : N nodes communite in rounds. omfected mpdes semdts to a random node 
Anti-Entropy: send the hash of the database 
- New node will merge unseen updates 

What is the communication graph? By looking into it we can design better protocol 






